### Authenticate Google Account 
if your google account is already authenticated, you can ignore.

In [ ]:
!pip install gcloud auth application-default login

### Import Libraries

In [1]:
import pandas as pd
import os
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

import warnings
warnings.filterwarnings("ignore")

### Accessing Google Cloud Storage

In [15]:
# Accessing Google Cloud Storage
project_id = 'gcp-data-analyst-1'  # Replace with your GCP project ID
bucket_name = 'data-ops-bucket-dev'  # Replace with your GCS bucket name

### Initialize storage bucket

In [16]:
# Initialize storage bucket
client = storage.Client(project=project_id)

# Access the bucket
bucket = client.bucket(bucket_name)

### List all the files in the bucket

In [17]:
def list_files_in_bucket():
    print(f"files in GCS Bucket '{bucket_name}':")
    blobs = bucket.list_blobs()
    for blob in blobs:
        print(f" - {blob.name}")

# List all the files in the bucket
list_files_in_bucket()

files in GCS Bucket 'data-ops-bucket-dev':
 - school_operations_adult_professionalism2.csv


#### Download a specific file from bucket

In [19]:
BUCKET_NAME = "data-ops-bucket-dev"
TARGET_PATH = r"C:\Users\SURESH KASIRABOINA\Downloads"
GCS_BUCKET_PATH = 'gs://data-ops-bucket-dev/school_operations_adult_professionalism2.csv'


# Bucket Configuration
bucket = client.bucket(BUCKET_NAME)

#function to 
def download_file(blob_name, destination_folder):
    os.makedirs(destination_folder, exist_ok = True)
    destination_path = os.path.join(destination_folder, os.path.basename(blob_name))

    blob = bucket.blob(blob_name)
    blob.download_to_filename(destination_path)

    print(f"DOWNLOAD FROM : '{blob_name}\n")
    print(f"TO DIRECTORY:'{destination_path}'")


# Call the function
download_file(GCS_BUCKET_PATH,TARGET_PATH)

NotFound: 404 GET https://storage.googleapis.com/download/storage/v1/b/data-ops-bucket-dev/o/gs%3A%2F%2Fdata-ops-bucket-dev%2Fschool_operations_adult_professionalism2.csv?alt=media: No such object: data-ops-bucket-dev/gs://data-ops-bucket-dev/school_operations_adult_professionalism2.csv: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)

In [20]:
from urllib.parse import urlparse

def get_blob_name(gcs_path):
    # Extract the object name from a GCS URI
    parsed = urlparse(gcs_path)
    if parsed.scheme == 'gs':
        return parsed.path.lstrip('/')
    return gcs_path

# Call the function with the correct blob name
blob_name = get_blob_name(GCS_BUCKET_PATH)
download_file(blob_name, TARGET_PATH)

DOWNLOAD FROM : 'school_operations_adult_professionalism2.csv

TO DIRECTORY:'C:\Users\SURESH KASIRABOINA\Downloads\school_operations_adult_professionalism2.csv'


In [6]:
TARGET_PATH = r"C:\Users\SURESH KASIRABOINA\Downloads"

### Accessing BigQuery to Read and write data

#### List all tables in BigQuery table

In [7]:
# List all tables in the dataset to verify table names

# Set your project and dataset info
PROJECT_ID = 'gcp-data-analyst-1'

dataset_id = f"{PROJECT_ID}.user_analytics_db"
bq_client = bigquery.Client(project=PROJECT_ID)

tables = list(bq_client.list_tables(dataset_id))
print(f"Tables in dataset '{dataset_id}':")
for table in tables:
    print(f" - {table.table_id}")

Tables in dataset 'gcp-data-analyst-1.user_analytics_db':
 - campaigns
 - customer_segments
 - daily_performance
 - employee_view
 - external_employee
 - sales_data
 - sales_data_part
 - stackoverflow
 - stackoverflow_post_question_part


#### Read BigQuery table

In [11]:
# Set your project and dataset info
PROJECT_ID = 'gcp-data-analyst-1'

def read_bigquery(query):

    client = bigquery.Client(project = PROJECT_ID)
    df = client.query(query).to_dataframe()
    
    print(f"Loaded {len(df)} rows from BigQuery.")
    return df


## Example usage:
sql_query = """
            SELECT * FROM gcp-data-analyst-1.user_analytics_db.campaigns LIMIT 10

            """
df = read_bigquery(sql_query)
df.head(5)

Loaded 1 rows from BigQuery.


,campaign_id,campaign_name,channel,start_date,end_date,budget,campaign_type,target_audience,geography
0,c004,Back to School,Instagram,2024-08-15,2024-09-15,40000.0,Social Media,Students,US


In [14]:
# group by lrl_brand and sum of sales
df_grouped = df.groupby('campaign_name')['budget'].sum().reset_index().rename(columns={'budget': 'total_budget'})
df_grouped = df_grouped.sort_values(by='total_budget', ascending=False).reset_index(drop=True)

df_grouped

,campaign_name,total_budget
0,Back to School,40000.0
